In [1]:
#!pip install rdflib
#!pip install rdfpandas
#!pip install rdflib-jsonld
#!pip install SPARQLWrapper

In [10]:
import rdflib
import pandas as pd
import json
#from collections import Counter
#from rdflib import Graph, plugin
#from rdflib.serializer import Serializer
import sddk
from ast import literal_eval
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from google.oauth2 import service_account # based on google-auth library

In [3]:
pd.options.display.max_columns = 999

In [4]:
# download 
!wget -O ../data/nomisma.rdf http://numismatics.org/ocre/nomisma.rdf

--2023-04-06 15:40:37--  http://numismatics.org/ocre/nomisma.rdf
Resolving numismatics.org (numismatics.org)... 18.191.94.77
Connecting to numismatics.org (numismatics.org)|18.191.94.77|:80... connected.
HTTP request sent, awaiting response... 200 
Length: unspecified [application/rdf+xml]
Saving to: ‘../data/nomisma.rdf’

../data/nomisma.rdf     [     <=>            ] 116.92M  7.61MB/s    in 16s     

2023-04-06 15:42:05 (7.32 MB/s) - ‘../data/nomisma.rdf’ saved [122601420]



In [5]:
%%time
g = rdflib.Graph()
g.parse('../data/nomisma.rdf')

Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#gYear, Converter=<function parse_date at 0x105a29e10>
Traceback (most recent call last):
  File "/Users/kasev/Projects/coins/coins_venv/lib/python3.10/site-packages/rdflib/term.py", line 2084, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "/Users/kasev/Projects/coins/coins_venv/lib/python3.10/site-packages/isodate/isodates.py", line 203, in parse_date
    raise ISO8601Error('Unrecognised ISO 8601 date format: %r' % datestring)
isodate.isoerror.ISO8601Error: Unrecognised ISO 8601 date format: '-0019'
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#gYear, Converter=<function parse_date at 0x105a29e10>
Traceback (most recent call last):
  File "/Users/kasev/Projects/coins/coins_venv/lib/python3.10/site-packages/rdflib/term.py", line 2084, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]

CPU times: user 41.3 s, sys: 1.57 s, total: 42.9 s
Wall time: 43.6 s


<Graph identifier=Naf711c8560374718b61c4bf06d8520a4 (<class 'rdflib.graph.Graph'>)>

In [3]:
%%time
# other option is to read the file directly from the source
#g = rdflib.Graph()
#g.parse("http://numismatics.org/ocre/nomisma.rdf")

<Graph identifier=N36edf143f03b4d94ae95412cedae417e (<class 'rdflib.graph.Graph'>)>

In [6]:
len(g) # 1288846

1288733

In [7]:
nomisma_json_bytes = g.serialize(format='json-ld', indent=4)

In [8]:
type(nomisma_json_bytes)

str

In [13]:
nomisma_json = json.loads(nomisma_json_bytes)

In [17]:
with open('../data/nomisma.json', 'w') as f:
    json.dump(nomisma_json, f)

In [18]:
nomisma_json[0]

{'@id': 'http://numismatics.org/ocre/id/ric.8.cnp.157',
 '@type': ['http://nomisma.org/ontology#TypeSeriesItem',
  'http://www.w3.org/2004/02/skos/core#Concept'],
 'http://nomisma.org/ontology#hasAuthority': [{'@id': 'http://nomisma.org/id/julian_the_apostate'}],
 'http://nomisma.org/ontology#hasDenomination': [{'@id': 'http://nomisma.org/id/solidus'}],
 'http://nomisma.org/ontology#hasEndDate': [{'@type': 'http://www.w3.org/2001/XMLSchema#gYear',
   '@value': '0363'}],
 'http://nomisma.org/ontology#hasManufacture': [{'@id': 'http://nomisma.org/id/struck'}],
 'http://nomisma.org/ontology#hasMaterial': [{'@id': 'http://nomisma.org/id/av'}],
 'http://nomisma.org/ontology#hasMint': [{'@id': 'http://nomisma.org/id/constantinople'}],
 'http://nomisma.org/ontology#hasObverse': [{'@id': 'http://numismatics.org/ocre/id/ric.8.cnp.157#obverse'}],
 'http://nomisma.org/ontology#hasRegion': [{'@id': 'http://nomisma.org/id/thrace'}],
 'http://nomisma.org/ontology#hasReverse': [{'@id': 'http://numism

In [19]:
len(nomisma_json) # previously we had 169803

169786

In [20]:
provenances_ids = [el["@id"] for el in nomisma_json if "#pro" in el["@id"]]
provenances_ids[:5]

['http://numismatics.org/ocre/symbol/monogram.ric.10.marcian.6#provenance',
 'http://numismatics.org/ocre/symbol/monogram.ric.10.anthemius.7#provenance',
 'http://numismatics.org/ocre/symbol/monogram.ric.10.marcian.5#provenance',
 'http://numismatics.org/ocre/symbol/monogram.ric.10.zenonis.1#provenance',
 'http://numismatics.org/ocre/symbol/monogram.ric.10.basiliscus_marcus.1#provenance']

# Extracting coins alone

In [21]:
# filter for coins only
with_object_type = [el for el in nomisma_json if "http://nomisma.org/ontology#representsObjectType" in el.keys()]
coins_json = [el for el in with_object_type if el["http://nomisma.org/ontology#representsObjectType"][0]["@id"] ==  'http://nomisma.org/id/coin']

In [22]:
len(coins_json)

54403

In [23]:
coins_json_df = pd.DataFrame(coins_json)
coins_json_df.head(5)

,@id,@type,http://nomisma.org/ontology#hasAuthority,http://nomisma.org/ontology#hasDenomination,http://nomisma.org/ontology#hasEndDate,http://nomisma.org/ontology#hasManufacture,http://nomisma.org/ontology#hasMaterial,http://nomisma.org/ontology#hasMint,http://nomisma.org/ontology#hasObverse,http://nomisma.org/ontology#hasRegion,http://nomisma.org/ontology#hasReverse,http://nomisma.org/ontology#hasStartDate,http://nomisma.org/ontology#representsObjectType,http://purl.org/dc/terms/source,http://rdfs.org/ns/void#inDataset,http://www.w3.org/2004/02/skos/core#definition,http://www.w3.org/2004/02/skos/core#prefLabel,http://purl.org/dc/terms/replaces,http://www.w3.org/2004/02/skos/core#broader,http://www.w3.org/2004/02/skos/core#exactMatch,http://purl.org/dc/terms/isReplacedBy,http://nomisma.org/ontology#hasIssuer
0,http://numismatics.org/ocre/id/ric.8.cnp.157,"[http://nomisma.org/ontology#TypeSeriesItem, h...",[{'@id': 'http://nomisma.org/id/julian_the_apo...,[{'@id': 'http://nomisma.org/id/solidus'}],[{'@type': 'http://www.w3.org/2001/XMLSchema#g...,[{'@id': 'http://nomisma.org/id/struck'}],[{'@id': 'http://nomisma.org/id/av'}],[{'@id': 'http://nomisma.org/id/constantinople'}],[{'@id': 'http://numismatics.org/ocre/id/ric.8...,[{'@id': 'http://nomisma.org/id/thrace'}],[{'@id': 'http://numismatics.org/ocre/id/ric.8...,[{'@type': 'http://www.w3.org/2001/XMLSchema#g...,[{'@id': 'http://nomisma.org/id/coin'}],[{'@id': 'http://nomisma.org/id/ric'}],[{'@id': 'http://numismatics.org/ocre/'}],"[{'@language': 'en', '@value': 'RIC VIII Const...","[{'@language': 'en', '@value': 'RIC VIII Const...",NaN,NaN,NaN,NaN,NaN
1,http://numismatics.org/ocre/id/ric.5.all.77,"[http://nomisma.org/ontology#TypeSeriesItem, h...",[{'@id': 'http://nomisma.org/id/allectus'}],[{'@id': 'http://nomisma.org/id/antoninianus'}],[{'@type': 'http://www.w3.org/2001/XMLSchema#g...,[{'@id': 'http://nomisma.org/id/struck'}],[{'@id': 'http://nomisma.org/id/ar'}],[{'@id': 'http://nomisma.org/id/camulodunum'}],[{'@id': 'http://numismatics.org/ocre/id/ric.5...,[{'@id': 'http://nomisma.org/id/britannia'}],[{'@id': 'http://numismatics.org/ocre/id/ric.5...,[{'@type': 'http://www.w3.org/2001/XMLSchema#g...,[{'@id': 'http://nomisma.org/id/coin'}],[{'@id': 'http://nomisma.org/id/ric'}],[{'@id': 'http://numismatics.org/ocre/'}],"[{'@language': 'en', '@value': 'RIC V Allectus...","[{'@language': 'en', '@value': 'RIC V Allectus...",NaN,NaN,NaN,NaN,NaN
2,http://numismatics.org/ocre/id/ric.2_3(2).hdn....,"[http://nomisma.org/ontology#TypeSeriesItem, h...",[{'@id': 'http://nomisma.org/id/hadrian'}],[{'@id': 'http://nomisma.org/id/sestertius'}],[{'@type': 'http://www.w3.org/2001/XMLSchema#g...,[{'@id': 'http://nomisma.org/id/struck'}],[{'@id': 'http://nomisma.org/id/orichalcum'}],[{'@id': 'http://nomisma.org/id/rome'}],[{'@id': 'http://numismatics.org/ocre/id/ric.2...,[{'@id': 'http://nomisma.org/id/italy'}],[{'@id': 'http://numismatics.org/ocre/id/ric.2...,[{'@type': 'http://www.w3.org/2001/XMLSchema#g...,[{'@id': 'http://nomisma.org/id/coin'}],[{'@id': 'http://nomisma.org/id/ric'}],[{'@id': 'http://numismatics.org/ocre/'}],"[{'@language': 'en', '@value': 'RIC II, Part 3...","[{'@language': 'en', '@value': 'RIC II, Part 3...",[{'@id': 'http://numismatics.org/ocre/id/ric.2...,[{'@id': 'http://numismatics.org/ocre/id/ric.2...,[{'@id': 'http://numismatics.org/ocre/id/ric.2...,NaN,NaN
3,http://numismatics.org/ocre/id/ric.3.ant.1405C_as,"[http://nomisma.org/ontology#TypeSeriesItem, h...",[{'@id': 'http://nomisma.org/id/antoninus_pius'}],[{'@id': 'http://nomisma.org/id/as'}],[{'@type': 'http://www.w3.org/2001/XMLSchema#g...,[{'@id': 'http://nomisma.org/id/struck'}],[{'@id': 'http://nomisma.org/id/ae'}],[{'@id': 'http://nomisma.org/id/rome'}],[{'@id': 'http://numismatics.org/ocre/id/ric.3...,[{'@id': 'http://nomisma.org/id/italy'}],[{'@id': 'http://numismatics.org/ocre/id/ric.3...,[{'@type': 'http://www.w3.org/2001/XMLSchema#g...,[{'@id': 'http://nomisma.org/id/coin'}],[{'@id': 'http://nomisma.o

# Extracting Obverse and Reverse

In [24]:
# filter for reverses only
coins_reverses_json = [el for el in nomisma_json if "#reverse" in el["@id"]]
len(coins_reverses_json)

51849

In [25]:
coins_reverses_df = pd.DataFrame(coins_reverses_json)
coins_reverses_df.columns = [col + "_reverse" for col in coins_reverses_df.columns]
coins_reverses_df.head(5)

,@id_reverse,http://nomisma.org/ontology#hasLegend_reverse,http://nomisma.org/ontology#hasPortrait_reverse,http://purl.org/dc/terms/description_reverse,http://nomisma.org/ontology#hasControlmark_reverse
0,http://numismatics.org/ocre/id/ric.6.lug.64#re...,[{'@value': 'GENIO POP-VLI ROMANI'}],[{'@id': 'http://collection.britishmuseum.org/...,"[{'@language': 'en', '@value': 'Genius, wearin...",NaN
1,http://numismatics.org/ocre/id/ric.9.anch.35B....,[{'@value': 'GLORIA RO-MANORVM'}],NaN,"[{'@language': 'en', '@value': 'Emperor advanc...",NaN
2,http://numismatics.org/ocre/id/ric.5.gall(1).1...,[{'@value': 'PROVIDENTIA AVGG'}],[{'@id': 'http://collection.britishmuseum.org/...,"[{'@language': 'en', '@value': 'Providentia, d...",NaN
3,http://numismatics.org/ocre/id/ric.4.sa.626b#r...,[{'@value': 'VIRTVS AVGVSTI S C'}],[{'@id': 'http://nomisma.org/id/severus_alexan...,"[{'@language': 'en', '@value': 'Severus Alexan...",NaN
4,http://numismatics.org/ocre/id/ric.8.sis.223#r...,[{'@value': 'FEL TEMP - REPARATIO'}],[{'@id': 'http://nomisma.org/id/constantius_ii'}],"[{'@language': 'en', '@value': 'Constantius II...",NaN


In [26]:
coins_reverses_df["@id_reverse"] = coins_reverses_df["@id_reverse"].apply(lambda x: x.rpartition("/")[2])

In [27]:
# filter for obverses only
coins_obverses_json = [el for el in nomisma_json if "#obverse" in el["@id"]]
len(coins_obverses_json)

51852

In [28]:
coins_obverses_df = pd.DataFrame(coins_obverses_json)
coins_obverses_df.columns = [col + "_obverse" for col in coins_obverses_df.columns]
coins_obverses_df.head(5)

,@id_obverse,http://nomisma.org/ontology#hasLegend_obverse,http://nomisma.org/ontology#hasPortrait_obverse,http://purl.org/dc/terms/description_obverse
0,http://numismatics.org/ocre/id/ric.7.tri.326#o...,[{'@value': 'CONSTANTINVS IVN NOB C'}],[{'@id': 'http://nomisma.org/id/constantine_ii'}],"[{'@language': 'en', '@value': 'Bust of Consta..."
1,http://numismatics.org/ocre/id/ric.4.mcs.187#o...,[{'@value': 'IMP CAES M OPEL SEV MACRINVS AVG'}],[{'@id': 'http://nomisma.org/id/macrinus'}],"[{'@language': 'en', '@value': 'Bust of Macrin..."
2,http://numismatics.org/ocre/id/ric.10.visi.373...,[{'@value': 'D N IVLIVS MAIORI-ANVS P F AVG'}],[{'@id': 'http://nomisma.org/id/majorian'}],"[{'@language': 'en', '@value': 'Bust of Majori..."
3,http://numismatics.org/ocre/id/ric.4.ge.125A#o...,[{'@value': 'P SEPTIMIVS GETA CAES'}],[{'@id': 'http://nomisma.org/id/geta'}],"[{'@language': 'en', '@value': 'Bust of Geta,b..."
4,http://numismatics.org/ocre/id/ric.2_3(2).hdn....,"[{'@language': 'la', '@value': 'IMP CAES TRAIA...",[{'@id': 'http://nomisma.org/id/hadrian'}],"[{'@language': 'en', '@value': 'Bust of Hadria..."


In [29]:
coins_obverses_df["@id_obverse"] = coins_obverses_df["@id_obverse"].apply(lambda x: x.rpartition("/")[2])

In [30]:
# "@id"
columns_id = ["http://nomisma.org/ontology#hasAuthority", 
              "http://nomisma.org/ontology#hasDenomination",
              "http://nomisma.org/ontology#hasManufacture",
              "http://nomisma.org/ontology#hasMaterial",
              "http://nomisma.org/ontology#hasMint",
              "http://nomisma.org/ontology#hasRegion",
              "http://purl.org/dc/terms/source",
              "http://rdfs.org/ns/void#inDataset",
              "http://www.w3.org/2004/02/skos/core#broader",
              "http://purl.org/dc/terms/replaces",
              "http://purl.org/dc/terms/isReplacedBy",
              "http://www.w3.org/2004/02/skos/core#exactMatch",
              "http://nomisma.org/ontology#hasIssuer",
              "http://nomisma.org/ontology#representsObjectType",
              
              "http://nomisma.org/ontology#hasObverse",
              "http://nomisma.org/ontology#hasReverse",
             
             "http://nomisma.org/ontology#hasPortrait_obverse",
              "http://nomisma.org/ontology#hasPortrait_reverse"
             ]
 
# "@value"
columns_value = ["http://www.w3.org/2004/02/skos/core#definition",
                 "http://www.w3.org/2004/02/skos/core#prefLabel",
                 "http://nomisma.org/ontology#hasEndDate", 
                 "http://nomisma.org/ontology#hasStartDate",
                 
                "http://nomisma.org/ontology#hasLegend_obverse",
                "http://nomisma.org/ontology#hasLegend_reverse",
                "http://purl.org/dc/terms/description_obverse",
                "http://purl.org/dc/terms/description_reverse",
                ]

def extract_id(cell):
    try: return cell[0]["@id"].rpartition("/")[2]
    except: return ""

for col in columns_id:
    try: coins_json_df[col] = coins_json_df[col].apply(extract_id)
    except: pass
    try: coins_obverses_df[col] = coins_obverses_df[col].apply(extract_id)
    except: pass
    try: coins_reverses_df[col] = coins_reverses_df[col].apply(extract_id)
    except: pass
    
def extract_value(cell):
    try: return cell[0]["@value"].rpartition("/")[2]
    except: return ""

for col in columns_value:
    try: coins_json_df[col] = coins_json_df[col].apply(extract_value) 
    except: pass
    try: coins_obverses_df[col] = coins_obverses_df[col].apply(extract_value) 
    except: pass
    try: coins_reverses_df[col] = coins_reverses_df[col].apply(extract_value) 
    except: pass



In [31]:
coins_obverses_df.head(5)

,@id_obverse,http://nomisma.org/ontology#hasLegend_obverse,http://nomisma.org/ontology#hasPortrait_obverse,http://purl.org/dc/terms/description_obverse
0,ric.7.tri.326#obverse,CONSTANTINVS IVN NOB C,constantine_ii,"Bust of Constantine II, helmeted, cuirassed, r..."
1,ric.4.mcs.187#obverse,IMP CAES M OPEL SEV MACRINVS AVG,macrinus,"Bust of Macrinus, radiate, draped, right"
2,ric.10.visi.3737#obverse,D N IVLIVS MAIORI-ANVS P F AVG,majorian,"Bust of Majorian, helmeted, rosette-diademed w..."
3,ric.4.ge.125A#obverse,P SEPTIMIVS GETA CAES,geta,"Bust of Geta,bare-headed, draped, cuirassed, r..."
4,ric.2_3(2).hdn.65#obverse,IMP CAES TRAIAN HADRIANO AVG DIVI TRA,hadrian,"Bust of Hadrian, laureate, cuirassed, right, v..."


In [32]:
coins_reverses_df.head(5)

,@id_reverse,http://nomisma.org/ontology#hasLegend_reverse,http://nomisma.org/ontology#hasPortrait_reverse,http://purl.org/dc/terms/description_reverse,http://nomisma.org/ontology#hasControlmark_reverse
0,ric.6.lug.64#reverse,GENIO POP-VLI ROMANI,58396,"Genius, wearing modius, nude, chlamys draped o...",NaN
1,ric.9.anch.35B.3#reverse,GLORIA RO-MANORVM,,"Emperor advancing right, dragging captive with...",NaN
2,ric.5.gall(1).158#reverse,PROVIDENTIA AVGG,59843,"Providentia, draped, standing left, holding ba...",NaN
3,ric.4.sa.626b#reverse,VIRTVS AVGVSTI S C,severus_alexander,"Severus Alexander (dressed as Romulus?), in mi...",NaN
4,ric.8.sis.223#reverse,FEL TEMP - REPARATIO,constantius_ii,"Constantius II, draped, cuirassed, standing le...",NaN


In [33]:
coins_json_df.rename(columns={"http://nomisma.org/ontology#hasObverse" : "@id_obverse"}, inplace=True)
coins_json_df.rename(columns={"http://nomisma.org/ontology#hasReverse" : "@id_reverse"}, inplace=True)


coins_json_df.head(5)

,@id,@type,http://nomisma.org/ontology#hasAuthority,http://nomisma.org/ontology#hasDenomination,http://nomisma.org/ontology#hasEndDate,http://nomisma.org/ontology#hasManufacture,http://nomisma.org/ontology#hasMaterial,http://nomisma.org/ontology#hasMint,@id_obverse,http://nomisma.org/ontology#hasRegion,@id_reverse,http://nomisma.org/ontology#hasStartDate,http://nomisma.org/ontology#representsObjectType,http://purl.org/dc/terms/source,http://rdfs.org/ns/void#inDataset,http://www.w3.org/2004/02/skos/core#definition,http://www.w3.org/2004/02/skos/core#prefLabel,http://purl.org/dc/terms/replaces,http://www.w3.org/2004/02/skos/core#broader,http://www.w3.org/2004/02/skos/core#exactMatch,http://purl.org/dc/terms/isReplacedBy,http://nomisma.org/ontology#hasIssuer
0,http://numismatics.org/ocre/id/ric.8.cnp.157,"[http://nomisma.org/ontology#TypeSeriesItem, h...",julian_the_apostate,solidus,0363,struck,av,constantinople,ric.8.cnp.157#obverse,thrace,ric.8.cnp.157#reverse,0361,coin,ric,,RIC VIII Constantinople 157,RIC VIII Constantinople 157,,,,,
1,http://numismatics.org/ocre/id/ric.5.all.77,"[http://nomisma.org/ontology#TypeSeriesItem, h...",allectus,antoninianus,0296,struck,ar,camulodunum,ric.5.all.77#obverse,britannia,ric.5.all.77#reverse,0293,coin,ric,,RIC V Allectus 77,RIC V Allectus 77,,,,,
2,http://numismatics.org/ocre/id/ric.2_3(2).hdn....,"[http://nomisma.org/ontology#TypeSeriesItem, h...",hadrian,sestertius,0133,struck,orichalcum,rome,ric.2_3(2).hdn.1640#obverse,italy,ric.2_3(2).hdn.1640#reverse,0130,coin,ric,,"RIC II, Part 3 (second edition) Hadrian 1640","RIC II, Part 3 (second edition) Hadrian 1640",ric.2.hdn.847f,ric.2_3(2).hdn.1635-1640,ric.2.hdn.847f,,
3,http://numismatics.org/ocre/id/ric.3.ant.1405C_as,"[http://nomisma.org/ontology#TypeSeriesItem, h...",antoninus_pius,as,0161,struck,ae,rome,ric.3.ant.1405C_as#obverse,italy,ric.3.ant.1405C_as#reverse,0145,coin,ric,,RIC III Antoninus Pius 1405C (as),RIC III Antoninus Pius 1405C (as),ric.3.ant.1405c,,,,
4,http://numismatics.org/ocre/id/ric.5.gall(1).255a,"[http://nomisma.org/ontology#TypeSeriesItem, h...",valerian,dupondius,0253,struck,ae,rome,ric.5.gall(1).255a#obverse,italy,ric.5.gall(1).255a#reverse,0253,coin,ric,,RIC V Gallienus (joint reign) 255a,RIC V Gallienus (joint reign) 255a,,ric.5.gall(1).255,,,


In [34]:
# test:
coins_json_df.merge(coins_obverses_df, on="@id_obverse").head(5)

,@id,@type,http://nomisma.org/ontology#hasAuthority,http://nomisma.org/ontology#hasDenomination,http://nomisma.org/ontology#hasEndDate,http://nomisma.org/ontology#hasManufacture,http://nomisma.org/ontology#hasMaterial,http://nomisma.org/ontology#hasMint,@id_obverse,http://nomisma.org/ontology#hasRegion,@id_reverse,http://nomisma.org/ontology#hasStartDate,http://nomisma.org/ontology#representsObjectType,http://purl.org/dc/terms/source,http://rdfs.org/ns/void#inDataset,http://www.w3.org/2004/02/skos/core#definition,http://www.w3.org/2004/02/skos/core#prefLabel,http://purl.org/dc/terms/replaces,http://www.w3.org/2004/02/skos/core#broader,http://www.w3.org/2004/02/skos/core#exactMatch,http://purl.org/dc/terms/isReplacedBy,http://nomisma.org/ontology#hasIssuer,http://nomisma.org/ontology#hasLegend_obverse,http://nomisma.org/ontology#hasPortrait_obverse,http://purl.org/dc/terms/description_obverse
0,http://numismatics.org/ocre/id/ric.8.cnp.157,"[http://nomisma.org/ontology#TypeSeriesItem, h...",julian_the_apostate,solidus,0363,struck,av,constantinople,ric.8.cnp.157#obverse,thrace,ric.8.cnp.157#reverse,0361,coin,ric,,RIC VIII Constantinople 157,RIC VIII Constantinople 157,,,,,,FL CL IVLIA-NVS PP AVG,julian_the_apostate,"Bust of Julian, pearl-diademed, draped, cuiras..."
1,http://numismatics.org/ocre/id/ric.5.all.77,"[http://nomisma.org/ontology#TypeSeriesItem, h...",allectus,antoninianus,0296,struck,ar,camulodunum,ric.5.all.77#obverse,britannia,ric.5.all.77#reverse,0293,coin,ric,,RIC V Allectus 77,RIC V Allectus 77,,,,,,IMP C ALLECTVS P F AVG,allectus,"Bust of Allectus, radiate, cuirassed, right"
2,http://numismatics.org/ocre/id/ric.2_3(2).hdn....,"[http://nomisma.org/ontology#TypeSeriesItem, h...",hadrian,sestertius,0133,struck,orichalcum,rome,ric.2_3(2).hdn.1640#obverse,italy,ric.2_3(2).hdn.1640#reverse,0130,coin,ric,,"RIC II, Part 3 (second edition) Hadrian 1640","RIC II, Part 3 (second edition) Hadrian 1640",ric.2.hdn.847f,ric.2_3(2).hdn.1635-1640,ric.2.hdn.847f,,,HADRIANVS AVG COS III P P,hadrian,"Bust of Hadrian, laureate, draped and cuirasse..."
3,http://numismatics.org/ocre/id/ric.3.ant.1405C_as,"[http://nomisma.org/ontology#TypeSeriesItem, h...",antoninus_pius,as,0161,struck,ae,rome,ric.3.ant.1405C_as#obverse,italy,ric.3.ant.1405C_as#reverse,0145,coin,ric,,RIC III Antoninus Pius 1405C (as),RIC III Antoninus Pius 1405C (as),ric.3.ant.1405c,,,,,FAVSTINA AVGVSTA AVG PII FIL,faustina_ii,"Bust of Faustina the Younger, draped, right, h..."
4,http://numismatics.org/ocre/id/ric.5.gall(1).255a,"[http://nomisma.org/ontology#TypeSeriesItem, h...",valerian,dupondius,0253,struck,ae,rome,ric.5.gall(1).255a#obverse,italy,ric.5.gall(1).255a#reverse,0253,coin,ric,,RIC V Gallienus (joint reign) 255a,RIC V Gallienus (joint reign) 255a,,ric.5.gall(1).255,,,,IMP C P LIC GALLIENVS AVG,gallienus,"Bust of Gallienus, radiate, draped, right"


In [35]:
coins_json_df = coins_json_df.merge(coins_obverses_df, on="@id_obverse")

In [36]:
# test
coins_json_df.merge(coins_reverses_df, on="@id_reverse").head(5)

,@id,@type,http://nomisma.org/ontology#hasAuthority,http://nomisma.org/ontology#hasDenomination,http://nomisma.org/ontology#hasEndDate,http://nomisma.org/ontology#hasManufacture,http://nomisma.org/ontology#hasMaterial,http://nomisma.org/ontology#hasMint,@id_obverse,http://nomisma.org/ontology#hasRegion,@id_reverse,http://nomisma.org/ontology#hasStartDate,http://nomisma.org/ontology#representsObjectType,http://purl.org/dc/terms/source,http://rdfs.org/ns/void#inDataset,http://www.w3.org/2004/02/skos/core#definition,http://www.w3.org/2004/02/skos/core#prefLabel,http://purl.org/dc/terms/replaces,http://www.w3.org/2004/02/skos/core#broader,http://www.w3.org/2004/02/skos/core#exactMatch,http://purl.org/dc/terms/isReplacedBy,http://nomisma.org/ontology#hasIssuer,http://nomisma.org/ontology#hasLegend_obverse,http://nomisma.org/ontology#hasPortrait_obverse,http://purl.org/dc/terms/description_obverse,http://nomisma.org/ontology#hasLegend_reverse,http://nomisma.org/ontology#hasPortrait_reverse,http://purl.org/dc/terms/description_reverse,http://nomisma.org/ontology#hasControlmark_reverse
0,http://numismatics.org/ocre/id/ric.8.cnp.157,"[http://nomisma.org/ontology#TypeSeriesItem, h...",julian_the_apostate,solidus,0363,struck,av,constantinople,ric.8.cnp.157#obverse,thrace,ric.8.cnp.157#reverse,0361,coin,ric,,RIC VIII Constantinople 157,RIC VIII Constantinople 157,,,,,,FL CL IVLIA-NVS PP AVG,julian_the_apostate,"Bust of Julian, pearl-diademed, draped, cuiras...",VIRTVS EXERCI-TVS ROMANORVM,,"Soldier, helmeted, draped, cuirassed, standing...",NaN
1,http://numismatics.org/ocre/id/ric.5.all.77,"[http://nomisma.org/ontology#TypeSeriesItem, h...",allectus,antoninianus,0296,struck,ar,camulodunum,ric.5.all.77#obverse,britannia,ric.5.all.77#reverse,0293,coin,ric,,RIC V Allectus 77,RIC V Allectus 77,,,,,,IMP C ALLECTVS P F AVG,allectus,"Bust of Allectus, radiate, cuirassed, right",LAETITI AVG,nomisma.rdf,"Laetitia, draped, standing left, holding wreat...",NaN
2,http://numismatics.org/ocre/id/ric.2_3(2).hdn....,"[http://nomisma.org/ontology#TypeSeriesItem, h...",hadrian,sestertius,0133,struck,orichalcum,rome,ric.2_3(2).hdn.1640#obverse,italy,ric.2_3(2).hdn.1640#reverse,0130,coin,ric,,"RIC II, Part 3 (second edition) Hadrian 1640","RIC II, Part 3 (second edition) Hadrian 1640",ric.2.hdn.847f,ric.2_3(2).hdn.1635-1640,ric.2.hdn.847f,,,HADRIANVS AVG COS III P P,hadrian,"Bust of Hadrian, laureate, draped and cuirasse...",S C (in field),nomisma.rdf,"Cappadocia, turreted, standing left, holding M...",NaN
3,http://numismatics.org/ocre/id/ric.3.ant.1405C_as,"[http://nomisma.org/ontology#TypeSeriesItem, h...",antoninus_pius,as,0161,struck,ae,rome,ric.3.ant.1405C_as#obverse,italy,ric.3.ant.1405C_as#reverse,0145,coin,ric,,RIC III Antoninus Pius 1405C (as),RIC III Antoninus Pius 1405C (as),ric.3.ant.1405c,,,,,FAVSTINA AVGVSTA AVG PII FIL,faustina_ii,"Bust of Faustina the Younger, draped, right, h...",S C,57039,"Diana, draped, standing front, head left, hold...",NaN
4,http://numismatics.org/ocre/id/ric.5.gall(1).255a,"[http://nomisma.org/ontology#TypeSeriesItem, h...",valerian,dupondius,0253,struck,ae,rome,ric.5.gall(1).255a#obverse,italy,ric.5.gall(1).255a#reverse,0253,coin,ric,,RIC V Gallienus (joint reign) 255a,RIC V Gallienus (joint reign) 255a,,ric.5.gall(1).255,,,,IMP C P LIC GALLIENVS AVG,gallienus,"Bust of Gallienus, radiate, draped, right",LIBERALITAS AVG S C,59099,"Liberalitas, draped, standing left, holding te...",NaN


In [37]:
coins_json_df = coins_json_df.merge(coins_reverses_df, on="@id_reverse")

In [38]:
shorter_col_names = [col.rpartition("/")[2] for col in coins_json_df.columns]
shorter_col_names

['@id',
 '@type',
 'ontology#hasAuthority',
 'ontology#hasDenomination',
 'ontology#hasEndDate',
 'ontology#hasManufacture',
 'ontology#hasMaterial',
 'ontology#hasMint',
 '@id_obverse',
 'ontology#hasRegion',
 '@id_reverse',
 'ontology#hasStartDate',
 'ontology#representsObjectType',
 'source',
 'void#inDataset',
 'core#definition',
 'core#prefLabel',
 'replaces',
 'core#broader',
 'core#exactMatch',
 'isReplacedBy',
 'ontology#hasIssuer',
 'ontology#hasLegend_obverse',
 'ontology#hasPortrait_obverse',
 'description_obverse',
 'ontology#hasLegend_reverse',
 'ontology#hasPortrait_reverse',
 'description_reverse',
 'ontology#hasControlmark_reverse']

In [39]:
coins_json_df.columns = shorter_col_names

In [40]:
coins_json_df.head(5)

,@id,@type,ontology#hasAuthority,ontology#hasDenomination,ontology#hasEndDate,ontology#hasManufacture,ontology#hasMaterial,ontology#hasMint,@id_obverse,ontology#hasRegion,@id_reverse,ontology#hasStartDate,ontology#representsObjectType,source,void#inDataset,core#definition,core#prefLabel,replaces,core#broader,core#exactMatch,isReplacedBy,ontology#hasIssuer,ontology#hasLegend_obverse,ontology#hasPortrait_obverse,description_obverse,ontology#hasLegend_reverse,ontology#hasPortrait_reverse,description_reverse,ontology#hasControlmark_reverse
0,http://numismatics.org/ocre/id/ric.8.cnp.157,"[http://nomisma.org/ontology#TypeSeriesItem, h...",julian_the_apostate,solidus,0363,struck,av,constantinople,ric.8.cnp.157#obverse,thrace,ric.8.cnp.157#reverse,0361,coin,ric,,RIC VIII Constantinople 157,RIC VIII Constantinople 157,,,,,,FL CL IVLIA-NVS PP AVG,julian_the_apostate,"Bust of Julian, pearl-diademed, draped, cuiras...",VIRTVS EXERCI-TVS ROMANORVM,,"Soldier, helmeted, draped, cuirassed, standing...",NaN
1,http://numismatics.org/ocre/id/ric.5.all.77,"[http://nomisma.org/ontology#TypeSeriesItem, h...",allectus,antoninianus,0296,struck,ar,camulodunum,ric.5.all.77#obverse,britannia,ric.5.all.77#reverse,0293,coin,ric,,RIC V Allectus 77,RIC V Allectus 77,,,,,,IMP C ALLECTVS P F AVG,allectus,"Bust of Allectus, radiate, cuirassed, right",LAETITI AVG,nomisma.rdf,"Laetitia, draped, standing left, holding wreat...",NaN
2,http://numismatics.org/ocre/id/ric.2_3(2).hdn....,"[http://nomisma.org/ontology#TypeSeriesItem, h...",hadrian,sestertius,0133,struck,orichalcum,rome,ric.2_3(2).hdn.1640#obverse,italy,ric.2_3(2).hdn.1640#reverse,0130,coin,ric,,"RIC II, Part 3 (second edition) Hadrian 1640","RIC II, Part 3 (second edition) Hadrian 1640",ric.2.hdn.847f,ric.2_3(2).hdn.1635-1640,ric.2.hdn.847f,,,HADRIANVS AVG COS III P P,hadrian,"Bust of Hadrian, laureate, draped and cuirasse...",S C (in field),nomisma.rdf,"Cappadocia, turreted, standing left, holding M...",NaN
3,http://numismatics.org/ocre/id/ric.3.ant.1405C_as,"[http://nomisma.org/ontology#TypeSeriesItem, h...",antoninus_pius,as,0161,struck,ae,rome,ric.3.ant.1405C_as#obverse,italy,ric.3.ant.1405C_as#reverse,0145,coin,ric,,RIC III Antoninus Pius 1405C (as),RIC III Antoninus Pius 1405C (as),ric.3.ant.1405c,,,,,FAVSTINA AVGVSTA AVG PII FIL,faustina_ii,"Bust of Faustina the Younger, draped, right, h...",S C,57039,"Diana, draped, standing front, head left, hold...",NaN
4,http://numismatics.org/ocre/id/ric.5.gall(1).255a,"[http://nomisma.org/ontology#TypeSeriesItem, h...",valerian,dupondius,0253,struck,ae,rome,ric.5.gall(1).255a#obverse,italy,ric.5.gall(1).255a#reverse,0253,coin,ric,,RIC V Gallienus (joint reign) 255a,RIC V Gallienus (joint reign) 255a,,ric.5.gall(1).255,,,,IMP C P LIC GALLIENVS AVG,gallienus,"Bust of Gallienus, radiate, draped, right",LIBERALITAS AVG S C,59099,"Liberalitas, draped, standing left, holding te...",NaN


In [41]:
len(coins_json_df)

51394

In [45]:
coins_json_df["@id"].str.contains("/ocre/").sum()

51394

In [42]:
coins_json_df.to_json("../data/coins_raw_df.json")

In [43]:
s = sddk.cloudSession("sciencedata.dk", "SDAM_root", "648597@au.dk")

connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/SDAM_root/


In [44]:
s.write_file("SDAM_data/coins/coins_df_2023.json", coins_json_df)

Your <class 'pandas.core.frame.DataFrame'> object has been succesfully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/coins/coins_df_2023.json"
